# **Convolutional Neural Networks and Computer Vision with TensorFlow**

Computer vision is the practice of writing algorithms which can discover patterns in visual data.

Such as the camera of a self-driving car recognizing the car in front.